In [ ]:
import json
import requests


def get_models(api_url):
    response = requests.get(api_url)

    if response.status_code == 200:
        return response.json()
    else:
        return {}

api_url = "http://localhost:1234/v1/models"
avaiable_models = get_models(api_url)

avaiable_models

In [54]:
import subprocess

def say_response(response_text, nombre, archivo="texto.txt"):
    # Escribir la respuesta en un archivo
    with open(archivo, "w", encoding="utf-8") as f:
        f.write(response_text)
    
    # Construir el comando
    comando = [
        "wine", "balcon/balcon.exe", 
        "-f", archivo, 
        "-n", nombre, 
        "-enc", "utf8",
    ]
    
    try:
        # Ejecutar el comando
        resultado = subprocess.run(comando, check=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    except subprocess.CalledProcessError as e:
        print(f"Error al ejecutar el comando: {e.stderr}")

In [ ]:
def launch_cli_interface():
    api_url = "http://localhost:1234/v1/chat/completions"

    messages = [
        {"role": "system", "content": "Eres mi asistente personal y waifu, debes responder como tal"},
    ]

    headers = {
        "Content-Type": "application/json"
    }

    user_input=""

    while user_input not in ["bye","goodbye","end"]:
        # Mensaje del usuario
        user_input = input("Ingresa una respuesta")
        # user_input = get_user_input()
        messages.append({"role":"user","content":user_input})

        # Request Body
        request_data = {
            "messages": messages,
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": True
        }

        response = requests.post(api_url, headers=headers, data=json.dumps(request_data), stream=True)


        if response.status_code == 200:
            
            # Mensajes esperados
            response_text=""
            response_tokens=[]

            # Si la respuesta es por streaming, la mostramos en tiempo real
            for line in response.iter_lines():
                if line:
                    try:
                        decoded_line = line.decode('utf-8')

                        if decoded_line.startswith('data: '):
                            content = decoded_line[6:].strip()
                            
                            if content == '[DONE]':
                                break
                            else:
                                try:
                                    token = json.loads(content)
                                    text = token["choices"][0]["delta"]["content"]
                                    #print(text,end="")

                                    response_tokens.append(token)
                                    response_text+=text

                                    
                                except json.JSONDecodeError as e:
                                    pass
                                    #print(f"Error al decodificar JSON: {e}")
                    except:
                        print("END")

            messages.append({"role": "assistant", "content": response_text})
            say_response(response_text,"Esperanza","response.txt")
        else:
            print(f"Error: {response.status_code}")

launch_cli_interface()